In [16]:
!pip install gdown beautifulsoup4 lxml pysentimiento faiss-cpu sentence-transformers mtranslate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Descargamos Dataset

In [7]:
import gdown
url = 'https://drive.google.com/drive/folders/1xjkuCdlnRkWMOrdbZ-i49K_oWMP8TP8Y'
gdown.download_folder(url, output='data', quiet=False, use_cookies=False)

Retrieving folder contents


Processing file 1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq bgg_database.csv
Processing file 1YCu3xhZq4C5dYyekiluMabwyWBqQyd2c IMDB-Movie-Data.csv


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1yIWOgUV5WyskQvmq48QvF2Lzr0LxpAdq
To: /Users/pepeargentoo/NLP_M/data/bgg_database.csv
100%|██████████| 1.83M/1.83M [00:00<00:00, 6.68MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YCu3xhZq4C5dYyekiluMabwyWBqQyd2c
To: /Users/pepeargentoo/NLP_M/data/IMDB-Movie-Data.csv
100%|██████████| 309k/309k [00:00<00:00, 2.84MB/s]
Download completed


['data/bgg_database.csv', 'data/IMDB-Movie-Data.csv']

# Scrapper De Libros

In [8]:
import requests
from bs4 import BeautifulSoup
import csv
import ssl
import concurrent.futures
import time

ssl._create_default_https_context = ssl._create_unverified_context

class ScrapperBooks:
    def __init__(self, url):
        self.url = url

    def get_books(self):
        response = requests.get(self.url)
        soup = BeautifulSoup(response.content, 'html.parser')
        div = soup.find_all('div', class_='page_content')[0]
        ol = div.find('ol')
        return ['https://www.gutenberg.org' + li.find('a')['href'] for li in ol.find_all('li')]

    def get_book_details(self, book_url):
        print(book_url)
        response = requests.get(book_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', class_='bibrec')
        details = {
            tr.th.get_text(strip=True): tr.td.get_text(strip=True)
            for tr in table.find_all('tr')
            if tr.th and tr.td and tr.th.get_text(strip=True) in {'Author', 'Title', 'Summary'}
        }
        details['Link'] = book_url
        return details

    def save_to_csv(self, data, filename='data/books.csv'):
        with open(filename, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=['Title', 'Author', 'Summary', 'Link'])
            writer.writeheader()
            writer.writerows(data)

    def run(self):
        start_time = time.time()
        books = self.get_books()
        with concurrent.futures.ThreadPoolExecutor() as executor:
            books_details = list(executor.map(self.get_book_details, books))
        self.save_to_csv(books_details)
        print(f"{len(books_details)} libros guardados en 'books.csv'.")
        elapsed_time = time.time() - start_time
        print(f"Tiempo total: {elapsed_time:.2f} segundos")


In [9]:
scraper = ScrapperBooks("https://www.gutenberg.org/browse/scores/top1000.php#books-last1")
scraper.run()

https://www.gutenberg.org/ebooks/84https://www.gutenberg.org/ebooks/1342

https://www.gutenberg.org/ebooks/2701
https://www.gutenberg.org/ebooks/1513
https://www.gutenberg.org/ebooks/25344
https://www.gutenberg.org/ebooks/100
https://www.gutenberg.org/ebooks/145
https://www.gutenberg.org/ebooks/2641
https://www.gutenberg.org/ebooks/11
https://www.gutenberg.org/ebooks/37106
https://www.gutenberg.org/ebooks/41
https://www.gutenberg.org/ebooks/67979
https://www.gutenberg.org/ebooks/16389
https://www.gutenberg.org/ebooks/6761
https://www.gutenberg.org/ebooks/394
https://www.gutenberg.org/ebooks/2160
https://www.gutenberg.org/ebooks/6593
https://www.gutenberg.org/ebooks/4085
https://www.gutenberg.org/ebooks/5197
https://www.gutenberg.org/ebooks/1259
https://www.gutenberg.org/ebooks/174
https://www.gutenberg.org/ebooks/2542
https://www.gutenberg.org/ebooks/345
https://www.gutenberg.org/ebooks/76
https://www.gutenberg.org/ebooks/64317
https://www.gutenberg.org/ebooks/25558
https://www.gutenbe

# Clasificador

In [10]:
from pysentimiento import create_analyzer

class EmotionAnalyzer:
    """
    Clase para analizar el estado de ánimo del usuario usando NLP avanzado.
    """

    def __init__(self):
        """Inicializa el analizador con el modelo pre-entrenado en español."""
        self.analyzer = create_analyzer(task="emotion", lang="es")

    def predict_emotion(self, text):
        """
        Predice la emoción del texto ingresado por el usuario.

        Args:
            text (str): Texto del usuario.

        Returns:
            str: Categoría de estado de ánimo.
        """
        result = self.analyzer.predict(text)
        return self._map_emotion(result.output)

    def _map_emotion(self, emotion):
        """
        Mapea las emociones del modelo a categorías personalizadas.

        Args:
            emotion (str): Emoción detectada por el modelo.

        Returns:
            str: Categoría de estado de ánimo personalizada.
        """
        emotion_map = {
            'joy': 'Alegre',
            'surprise': 'Alegre',
            'sadness': 'Melancólico',
            'anger': 'Melancólico',
            'fear': 'Melancólico',
            'disgust': 'Melancólico',
            'neutral': 'Ni fu ni fa',
            'others': 'Ni fu ni fa'
        }
        return emotion_map.get(emotion, "Desconocido")

In [19]:
import pandas as pd
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from mtranslate import translate

class RecomendadorFAISS:
    def __init__(self, datasets):
        self.modelo = SentenceTransformer('paraphrase-MiniLM-L6-v2')
        self.resumidor = pipeline("summarization", model="facebook/bart-large-cnn")
        self.index = None
        self.data = []
        self._cargar_y_indexar_datasets(datasets)

    def _cargar_y_indexar_datasets(self, datasets):
        embeddings = []
        for ruta, columnas, columna_embed, fuente in datasets:
            df = pd.read_csv(ruta, usecols=columnas)
            df['Fuente'] = fuente  # Agregar la fuente como columna
            textos = df[columna_embed].fillna("").tolist()
            emb = self.modelo.encode(textos, convert_to_tensor=False)
            embeddings.extend(emb)
            self.data.extend(df.to_dict('records'))
        embeddings = np.array(embeddings, dtype='float32')
        self._crear_indice(embeddings)

    def _crear_indice(self, embeddings):
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(embeddings)

    def recomendar(self, preferencia, emocion):
        preferencia_en = translate(preferencia, "en")
        emocion_en = translate(emocion, "en")
        preferencia_embed = self.modelo.encode([preferencia_en], convert_to_tensor=False)
        _, indices = self.index.search(np.array(preferencia_embed, dtype='float32'), 3)
        return [self._formatear_recomendacion(self.data[i], emocion_en) for i in indices[0]]

    def _formatear_recomendacion(self, item, emocion):
        fuente = item['Fuente']
        titulo = item.get('Title', item.get('game_name', 'Sin título'))
        titulo_es = translate(titulo, "es")  # Traducción del título
        descripcion = item.get('description', item.get('Summary', 'Sin descripción'))
        resumen = self._resumir_y_traducir(descripcion)
        mensaje = f"{titulo_es} - {resumen} (Fuente: {fuente})"
        if emocion == 'happy':
            return f"¡Disfruta esto! {mensaje}"
        elif emocion == 'sad':
            return f"Esto podría animarte: {mensaje}"
        else:
            return f"Interesante opción: {mensaje}"

    def _resumir_y_traducir(self, texto):
        if len(texto.split()) > 50:
            resumen = self.resumidor(texto, max_length=50, min_length=25, do_sample=False)[0]['summary_text']
            return translate(resumen, "es")  # Traducción del resumen
        return translate(texto, "es")

datasets = [
    ('data/bgg_database.csv', ['game_name', 'description'], 'description', 'Juego'),
    ('data/books.csv', ['Title', 'Summary'], 'Summary', 'Libro'),
    ('data/IMDB-Movie-Data.csv', ['Title', 'Description'], 'Description', 'Película')
]

recomendador = RecomendadorFAISS(datasets)

respuesta = input("Describe tu día en una frase: ")
emotion = EmotionAnalyzer()  # Asume que esta clase ya está implementada
emocion_detectada = emotion.predict_emotion(respuesta)

preferencia = input("¿Qué temática te gustaría explorar? ")
recomendaciones = recomendador.recomendar(preferencia, emocion_detectada)

print(f"Estado emocional detectado: {emocion_detectada}")
print("Recomendaciones:")
print("\n" + "-" * 40)
for rec in recomendaciones:
    print(rec)
    print("-" * 40)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Estado emocional detectado: Ni fu ni fa
Recomendaciones:

----------------------------------------
Interesante opción: Hermosas historias de Shakespeare - "Beautiful Stories from Shakespeare" de E. Nesbit y William Shakespeare es una colección de versiones de las obras de Shakespeare. Este volumen tiene como objetivo presentar los encantadores cuentos que se encuentran en las obras de Shakespeare de una manera más sencilla y (Fuente: Libro)
----------------------------------------
Interesante opción: Las obras de Sir Thomas Browne, volumen 1 - "Las obras de Sir Thomas Browne, volumen 1" es una colección completa de textos literarios y filosóficos. El volumen incluye obras como "Religio Medici", una reflexión sobre la fe personal del autor y sus opiniones sobre la religión y la (Fuente: Libro)
----------------------------------------
Interesante opción: Las obras de Edgar Allan Poe — Volumen 4 - "Las obras de Edgar Allan Poe — Volumen 4" es una colección de piezas literarias probablemen